# Топ рекламодателей по рейтингу
Пример расчета топ-100 рекламодателей за период по рейтингу

## Описание задачи и условий расчета
- Период: 01.06.2023-30.06.2023
- ЦА: Все 18+, Все 4+
- Место просмотра: Все места (Дом+Дача)
- Каналы: все каналы проекта TV Index
- Ролики: ролик статус - Реальный
- Статистики: Rtg% (тотал)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике демографических атрибутов найдем нужные нам переменные
cats.get_tv_demo_attribute(names=['возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: age

In [ ]:
# Обратимся к словарю статусов события
cats.get_tv_issue_status()

# Для "Реальный" будем использовать id R

## Формирование задания
Зададим условия расчета

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2023-06-01', '2023-06-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = 'tvNetId IN (1,2,4,10,11,12,13,16,40,60,83,84,86,204,205,206,255,257,258,259,260,286,326,329,340,356,376,393,502,545,348)'

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов: статус - реальный и типы роликов классической базы
ad_filter = 'adIssueStatusId = R and adTypeId IN (1,10,23,25,5,15,24)'

# Указываем список срезов (группировок)
slices = [
    'advertiserName' #Рекламодатель
]

# Указываем список статистик для расчета
statistics = ['SpotByBreaksRtgPerSum']

# Задаем условия сортировки: Телекомпания (от а до я), рекламодатель (от а до я), Бренд (от а до я)
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "issueType": "AD" #Тип события - Ролики
}

Сформируем словарь с целевыми аудиториями

In [ ]:
# Задаем необходимые группы
targets = {
    'a. Все 18+':'age >= 18',
    'b. Все 4+':'age >= 4'
}

## Расчет задания

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задание и отправляем на расчет
for target, syntax in targets.items():
    
    # Подставляем значения словаря в параметры
    project_name = target 
    basedemo_filter = syntax
      
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_crosstab_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_crosstab_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[['prj_name']+slices+statistics]

df

In [ ]:
# Формируем сводную таблицу: строки - срезы, столбцы - ЦА, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=slices, 
                        columns=['prj_name'])
df

In [ ]:
#Сортируем по убыванию рейтинга 4+
df.sort_values([('SpotByBreaksRtgPerSum', 'b. Все 4+')], ascending=False, inplace=True)

# Выводим первые 100 строк
df = df.head(100)

df

## Сохраняем в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('07_top100_advertisers')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. Можно указать несколько периодов
date_filter = [('2023-06-01', '2023-06-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем ЦА
basedemo_filter = 'age >= 18'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter=None

# Задаем каналы
company_filter = 'tvNetId IN (1,2,4,10,11,12,13,16,40,60,83,84,86,204,205,206,255,257,258,259,260,286,326,329,340,356,376,393,502,545,348)'

# Указываем фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов: статус - реальный и типы роликов классической базы
ad_filter = 'adIssueStatusId = R and adTypeId IN (1,10,23,25,5,15,24) and advertiserId = 1110'

# Указываем список срезов (группировок)
slices = [
    'adSpotId', #Программа ID выхода, обязательный атрибут! 
    'researchDate', #Дата, обязательный атрибут! 
    'tvCompanyName',
    'advertiserName' #Рекламодатель
]

# Указываем список статистик для расчета
statistics = ['SpotByBreaksRtgPer']

# Задаем условия сортировки: Телекомпания (от а до я), рекламодатель (от а до я), Бренд (от а до я)
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
}

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_simple_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                        daytype_filter=daytype_filter, company_filter=company_filter, 
                                        location_filter=location_filter, basedemo_filter=basedemo_filter, 
                                        targetdemo_filter=targetdemo_filter,program_filter=program_filter, 
                                        break_filter=break_filter, ad_filter=ad_filter, 
                                        slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_simple_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

In [ ]:
# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]
df

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('07_top100_advertisers')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)